In [1]:
from datetime import datetime, timedelta
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import time

def plotWithInterpolatedRegression(path, name, annotation=None):
    # Load data from path
    with open(path) as file:
        log = json.load(file)
    
    # Convert columns to numpy arrays
    dates = np.array([entry['date'] for entry in log])
    estimated_maxes = np.array([entry['estimatedMax'] for entry in log])

    # Convert dates to datetime objects for proper plotting
    date_objects = [datetime.strptime(date, "%m/%d/%Y") for date in dates]
    
    # Fill in missing dates
    start_date = min(date_objects)
    end_date = max(date_objects)
    date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    date_range_timestamps = [date.timestamp() for date in date_range]
    
    # Interpolate estimated max values
    date_timestamps = [date.timestamp() for date in date_objects]
    interpolation_function = interp1d(date_timestamps, estimated_maxes, kind='cubic')
    estimated_max_interpolated = interpolation_function(date_range_timestamps)
        
    # Perform polynomial regression of degree 3
    regression_coeffs = np.polyfit(date_range_timestamps, estimated_max_interpolated, 3)
    regression = np.poly1d(regression_coeffs)
    regression_values = regression(date_range_timestamps)

    # Create the line chart
    plt.figure(figsize=(10, 6))
    plt.plot(date_objects, estimated_maxes, marker='o')
    plt.plot(date_range, regression_values, color='red', linestyle='dashed', label='Polynomial Regression')
    plt.title(f'{name} - Estimated Max Progression')
    plt.xlabel("Date")
    plt.ylabel("Estimated Max")
    plt.xticks(rotation=45)
    
    # Optional annotation
    if annotation:
        plt.annotate(annotation[0], xy=annotation[1], xycoords='axes fraction', fontsize=10, ha='center')
        
    plt.tight_layout()

    # Display the chart
    plt.show()